<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_FvS_QoQ_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 624 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
# change this
curr_quarter = 'Sep 2022'
prev_quarter = 'Jun 2022'
len(stock_ids)

63

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if curr_quarter in test_list:
        quarter = curr_quarter
    else:
        quarter = prev_quarter

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-7)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

ASIANPAINT
172
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-0

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
7,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15
8,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04
9,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44
10,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96
11,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51
...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11


In [5]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
7,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,5.93,36.38,5585.0,82.15,2988.70,-6.73,11.23,4.50,27.05,4.72,4.50
9,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
10,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
11,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51,8.87,31.60,7893.0,96.96,3064.00,14.78,-2.90,11.88,9.05,19.50,11.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11,1.89,10.04,2323.0,23.83,239.25,-11.16,12.23,1.07,-20.53,-41.27,1.07


In [6]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2022-08-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,INDIGO.NS,2022-08-15,60.27,-36.72,23.99
11,BERGEPAINT.NS,2022-08-15,26.14,14.98,3.05
11,PIDILITIND.NS,2022-08-15,23.69,39.20,23.59
11,IRCTC.NS,2022-08-15,23.44,14.98,2.26
11,TATAPOWER.NS,2022-08-15,21.20,58.60,4.29
11,TITAN.NS,2022-08-15,21.13,54.01,18.16
11,NAUKRI.NS,2022-08-15,15.64,-71.60,27.99
11,DMART.NS,2022-08-15,14.25,50.68,34.23
11,SUNPHARMA.NS,2022-08-15,13.92,-190.52,3.50
11,HEROMOTOCO.NS,2022-08-15,12.69,-3.43,12.75


In [10]:
df_analysis_final.to_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100_FvS_QoQ_Analysis.csv', index=False)


In [12]:
df_growth = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,3IINFOLTD.NS,2020-11-15,32.00,0.09,152.0,0.47,747.0,68.09,0.06,0.46,150.0,61.96,28.50,2.17,10.11,12.28,1.33,50.00,12.28
2,3IINFOLTD.NS,2021-02-15,69.00,0.14,158.0,0.44,618.0,156.82,0.09,0.47,152.0,68.09,32.00,-6.38,122.00,115.62,3.95,55.56,115.62
3,3IINFOLTD.NS,2021-05-16,92.50,2.12,149.0,2.41,609.0,38.38,0.14,0.44,158.0,156.82,69.00,447.73,-413.68,34.05,-5.70,1414.29,34.05
4,3IINFOLTD.NS,2021-08-15,79.50,-0.05,160.0,2.30,619.0,34.57,2.12,2.41,149.0,38.38,92.50,-4.56,-9.47,-14.03,7.38,-102.36,-14.03
5,3IINFOLTD.NS,2021-11-15,70.05,-1.43,177.0,0.78,644.0,89.81,-0.05,2.30,160.0,34.57,79.50,-66.09,54.20,-11.89,10.62,2760.00,-11.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,ZEEL.NS,2020-11-15,193.10,0.98,1723.0,-3.04,7035.0,-63.52,0.32,0.28,1312.0,575.71,161.20,-1185.71,1205.50,19.79,31.33,206.25,19.79
237,ZEEL.NS,2021-02-15,212.50,4.16,2729.0,-2.52,7715.0,-84.33,0.98,-3.04,1723.0,-63.52,193.10,-17.11,27.16,10.05,58.39,324.49,10.05
238,ZEEL.NS,2021-05-16,191.60,2.87,1966.0,8.33,7730.0,23.00,4.16,-2.52,2729.0,-84.33,212.50,-430.56,420.73,-9.83,-27.96,-31.01,-9.83
239,ZEEL.NS,2021-08-15,183.25,2.23,1775.0,10.24,8193.0,17.90,2.87,8.33,1966.0,23.00,191.60,22.93,-27.26,-4.33,-9.72,-22.30,-4.33


In [13]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

TATASTEEL.NS     4
MOTILALOFS.NS    4
ZEEL.NS          3
MINDTREE.NS      3
DMART.NS         3
TIPSINDLTD.NS    3
BERGEPAINT.NS    3
HAVELLS.NS       3
SAIL.NS          3
IRCTC.NS         3
ALKYLAMINE.NS    3
NATIONALUM.NS    3
TITAN.NS         2
TATAPOWER.NS     2
RELIANCE.NS      2
NAUKRI.NS        2
INFY.NS          2
IEX.NS           2
HEROMOTOCO.NS    2
DIVISLAB.NS      2
Name: Stock, dtype: int64

In [14]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [15]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
222,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
143,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
161,MINDTREE.NS,2021-11-15,12.83,16.12,68.22,0,0,0.0,3.0,3.0
119,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
186,SAIL.NS,2021-11-15,29.96,11.23,-12.47,0,0,4.0,1.0,5.0
228,TIPSINDLTD.NS,2021-11-15,242.86,44.22,26.53,0,0,3.0,4.0,7.0
5,3IINFOLTD.NS,2021-11-15,10.62,2760.00,-11.89,0,0,3.0,4.0,7.0
137,NAUKRI.NS,2021-11-15,11.31,4738.29,18.12,0,0,1.0,7.0,8.0
167,MOTILALOFS.NS,2021-11-15,47.39,142.36,17.78,0,0,4.0,4.0,8.0
173,NATIONALUM.NS,2021-11-15,45.13,115.34,21.24,0,0,4.0,4.0,8.0


In [16]:
df_tmp_final.sort_values(by = 'total_weight', ascending=True).head(20)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
222,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
161,MINDTREE.NS,2021-11-15,12.83,16.12,68.22,0,0,0.0,3.0,3.0
143,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
77,HCLTECH.NS,2021-11-15,2.93,1.69,4.74,0,0,0.0,3.0,3.0
198,TCS.NS,2021-11-15,3.21,6.86,1.82,0,0,0.0,4.0,4.0
59,DIVISLAB.NS,2021-11-15,1.38,8.81,-2.12,0,0,2.0,2.0,4.0
119,IEX.NS,2021-11-15,20.88,22.86,93.71,0,0,4.0,0.0,4.0
186,SAIL.NS,2021-11-15,29.96,11.23,-12.47,0,0,4.0,1.0,5.0
5,3IINFOLTD.NS,2021-11-15,10.62,2760.00,-11.89,0,0,3.0,4.0,7.0
228,TIPSINDLTD.NS,2021-11-15,242.86,44.22,26.53,0,0,3.0,4.0,7.0


In [17]:
scrip = 'DIVISLAB.NS'
df_growth[df_growth['Stock'] == scrip][cols]

,Stock,Quarter,Sales%,EPS%,Return%
55,DIVISLAB.NS,2020-11-15,1.10,5.56,11.48
56,DIVISLAB.NS,2021-02-15,-2.74,-9.40,7.98
57,DIVISLAB.NS,2021-05-16,5.11,6.66,8.10
58,DIVISLAB.NS,2021-08-15,9.68,11.00,23.08
59,DIVISLAB.NS,2021-11-15,1.38,8.81,-2.12
